In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train_path = "./input/train_local.csv"
test_path = "./input/test_local.csv"

In [3]:
import hashlib

In [4]:
s="A"
int(hashlib.sha256(s.encode('utf-8')).hexdigest(), 16) % 10**2

45

In [5]:
def hash_cat(s):
    return int(hashlib.sha256(s.encode('utf-8')).hexdigest(), 16) % 10**3

In [6]:
# x = ['A', "b"]


In [7]:
# x.index('c')

In [8]:
cat_dict = []
def hash_cat_two(s):
    if s not in cat_dict:
        cat_dict.append(s)
    return cat_dict.index(s)

In [9]:
import pandas as pd

In [10]:
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)


In [11]:
len(df_train)

240000

In [12]:
df_train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,target
0,495701,A,J,A,A,F,O,C,E,AX,...,0.387487,0.251159,0.801758,0.553776,0.444780,0.335810,0.345248,0.470157,0.319817,0
1,475514,A,N,A,A,G,BI,E,AV,AG,...,0.592863,0.250877,0.292144,0.358930,0.562570,0.707359,0.632267,0.284889,0.518947,0
2,289595,B,F,D,A,E,BI,A,F,L,...,0.835299,0.837267,0.357749,0.678743,0.575143,0.754098,0.642640,0.958589,0.940756,0
3,290200,B,I,A,A,E,BI,C,K,BM,...,0.369965,0.573459,0.455696,0.800336,0.608690,0.261167,0.493506,0.443034,0.342730,0
4,3547,A,B,D,A,F,BI,A,AS,AX,...,0.694353,0.577407,0.721532,0.487756,0.610030,0.668544,0.360062,0.763265,0.265792,0


In [13]:
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

In [14]:
df_train.columns

Index(['id', 'cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7',
       'cat8', 'cat9', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15',
       'cat16', 'cat17', 'cat18', 'cont0', 'cont1', 'cont2', 'cont3', 'cont4',
       'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'target'],
      dtype='object')

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
# ?train_test_split

In [17]:
cat_col = ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7',
       'cat8', 'cat9', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15',
       'cat16', 'cat17', 'cat18']
cont_col = ['cont0', 'cont1', 'cont2', 'cont3', 'cont4',
       'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10']

In [18]:
for col in cat_col:
    encoder = LabelEncoder()
    # encoder.fit(list(df_train[col]) + list(df_test[col]))
    encoder.fit(list(df_train[col].values) + list(df_test[col].values))
    df_train[col] = encoder.transform(df_train[col].values)
    df_test[col] = encoder.transform(df_test[col].values)
    print("done ", col)

done  cat0
done  cat1
done  cat2
done  cat3
done  cat4
done  cat5
done  cat6
done  cat7
done  cat8
done  cat9
done  cat10
done  cat11
done  cat12
done  cat13
done  cat14
done  cat15
done  cat16
done  cat17
done  cat18


In [19]:
X = df_train.drop(["id", "target"], axis=1)
y = df_train["target"]


In [20]:
X.shape

(240000, 30)

In [21]:
y.shape

(240000,)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=111)

In [23]:
import lightgbm as gbm

In [24]:
df_train["cat0"].unique()

array([0, 1])

In [25]:
train_data = gbm.Dataset(X_train, label=y_train, categorical_feature=cat_col)

In [26]:
validation_data = gbm.Dataset(X_test, label=y_test, reference=train_data)

In [27]:
lgbm_params = {
    'boosting': 'gbdt',          # dart (drop out trees) often performs better
    'application': 'binary',     # Binary classification
    'learning_rate': 0.02,       # Learning rate, controls size of a gradient descent step
    'min_data_in_leaf': 20,      # Data set is quite small so reduce this a bit
    'feature_fraction': 0.7,     # Proportion of features in each boost, controls overfitting
    'num_leaves': 100,            # Controls size of tree since LGBM uses leaf wise splits
    'metric': 'auc',  # Area under ROC curve as the evaulation metric
}

https://www.kaggle.com/sayantansadhu/lgbm-with-hyper-parameter-tuning-using-optuna

In [28]:
lgbm_params = {
    'boosting': 'gbdt',          # dart (drop out trees) often performs better
    'application': 'binary',     # Binary classification
    'learning_rate': 0.02,       # Learning rate, controls size of a gradient descent step
    'min_data_in_leaf': 100,      # Data set is quite small so reduce this a bit
    'feature_fraction': 0.7,     # Proportion of features in each boost, controls overfitting
    'metric': 'binary_logloss',  # Area under ROC curve as the evaulation metric,
    'lambda_l1': 5e-05, 
     'lambda_l2': 1.35e-08, 
     'num_leaves': 216, 
     'feature_fraction': 0.7458519562366559, 
     'bagging_fraction': 0.7835116194444349, 
     'bagging_freq': 2, 
     'min_child_samples': 95
}

# {'lambda_l1': 5.7529066102502985e-05, 
#  'lambda_l2': 1.3522520634637266e-08, 
#  'num_leaves': 216, 
#  'feature_fraction': 0.7458519562366559, 
#  'bagging_fraction': 0.7835116194444349, 
#  'bagging_freq': 2, 
#  'min_child_samples': 95}

In [29]:
num_round = 1000
bst = gbm.train(lgbm_params, train_data, num_round, valid_sets=[validation_data], early_stopping_rounds=10, verbose_eval=100)

/home/cpu11453/miniconda3/envs/nlp/lib/python3.7/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/cpu11453/miniconda3/envs/nlp/lib/python3.7/site-packages/lightgbm/basic.py:1555: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat0', 'cat1', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15', 'cat16', 'cat17', 'cat18', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=95 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=95 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Info] Number of positive: 57309, number of negative: 158691
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022074 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3390
[LightGBM] [Info] Number of data points in the train set: 216000, number of used features: 30
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=95 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.265319 -> initscore=-1.018501
[LightGBM] [Info] Start training from score -1.018501


/home/cpu11453/miniconda3/envs/nlp/lib/python3.7/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/cpu11453/miniconda3/envs/nlp/lib/python3.7/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 10 rounds
[100]	valid_0's binary_logloss: 0.365358
[200]	valid_0's binary_logloss: 0.347848
[300]	valid_0's binary_logloss: 0.344195
[400]	valid_0's binary_logloss: 0.342793
Early stopping, best iteration is:
[453]	valid_0's binary_logloss: 0.34251


In [30]:
# ?gbm.train

In [31]:
# df_test = pd.read_csv(test_path)
# df_test.head()

In [32]:
df_test2 = df_test.drop(["id", "target"], axis=1)

In [33]:
# for col in cat_col:
#     df_test2[col] = df_test2[col].apply(hash_cat_two)
#     print("done ", col)

In [34]:
# ?bst

In [35]:
df_test2.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10
0,0,5,0,0,6,33,0,14,38,4,...,0.616445,0.637685,0.301544,0.790893,0.488015,0.614380,0.539349,0.411866,0.661432,0.486744
1,0,5,2,0,4,33,2,11,23,0,...,0.350460,0.324491,0.349067,0.786160,0.554579,0.486805,0.390730,0.375106,0.569209,0.459002
2,0,13,0,0,4,33,2,30,38,9,...,0.323298,0.279434,0.346821,0.208250,0.618792,0.494801,0.367308,0.387041,0.474525,0.402563
3,0,8,0,0,4,33,0,39,5,4,...,0.331975,0.305931,0.321362,0.596346,0.797107,0.575397,0.326885,0.353614,0.258005,0.357856
4,0,12,2,0,4,33,6,30,38,0,...,0.240007,0.190550,0.578725,0.777659,0.773500,0.703389,0.239250,0.242388,0.283970,0.311369


In [36]:
pred = bst.predict(df_test2, num_iteration=bst.best_iteration)

In [37]:
df_test2

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10
0,0,5,0,0,6,33,0,14,38,4,...,0.616445,0.637685,0.301544,0.790893,0.488015,0.614380,0.539349,0.411866,0.661432,0.486744
1,0,5,2,0,4,33,2,11,23,0,...,0.350460,0.324491,0.349067,0.786160,0.554579,0.486805,0.390730,0.375106,0.569209,0.459002
2,0,13,0,0,4,33,2,30,38,9,...,0.323298,0.279434,0.346821,0.208250,0.618792,0.494801,0.367308,0.387041,0.474525,0.402563
3,0,8,0,0,4,33,0,39,5,4,...,0.331975,0.305931,0.321362,0.596346,0.797107,0.575397,0.326885,0.353614,0.258005,0.357856
4,0,12,2,0,4,33,6,30,38,0,...,0.240007,0.190550,0.578725,0.777659,0.773500,0.703389,0.239250,0.242388,0.283970,0.311369
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,1,8,0,0,7,33,0,14,3,4,...,0.699535,0.687533,0.737377,0.206023,0.620642,0.563338,0.735728,0.500539,0.306823,0.620584
59996,0,1,12,0,5,2,2,8,48,0,...,0.332429,0.280286,0.301266,0.248114,0.553265,0.327081,0.338067,0.381465,0.751954,0.330403
59997,0,10,6,0,3,33,0,8,51,0,...,0.354513,0.300859,0.302043,0.247338,0.104145,0.429906,0.278451,0.339213,0.384370,0.368506
59998,0,7,0,0,5,33,0,6,23,0,...,0.369078,0.420645,0.355458,0.202159,0.736260,0.489391,0.351315,0.610873,0.621696,0.454686


In [38]:
df_pred_final = df_test[["id"]]

In [39]:
df_pred_final["target"] = pred

/home/cpu11453/miniconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [40]:
df_pred_final.head()

,id,target
0,150904,0.042360
1,168869,0.056568
2,125842,0.108471
3,37203,0.146189
4,353336,0.405677


# df_pred_final.to_csv('submission.csv', index=False)

In [42]:
pred

array([0.04236017, 0.05656821, 0.10847092, ..., 0.08119716, 0.88511606,
       0.02479363])

# evaluation local

In [43]:
from sklearn.metrics import roc_auc_score

In [44]:
y_test_label = df_test["target"]

In [45]:
roc_auc_score(y_test_label, pred)

0.8969435887822527

In [46]:
# local test result 
auc = 0.8969435887822527